In [ ]:
import warnings
import torch
import plotly.express as px
!pip3 install sentence-transformers
from sentence_transformers import SentenceTransformer
warnings.filterwarnings('ignore')

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")
model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
tokenized_data = model.tokenize(["Silver skin laced with his golden blood"])

In [ ]:
tokenized_data

{'input_ids': tensor([[  101,  3165,  3096, 17958,  2007,  2010,  3585,  2668,   102]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
model.tokenizer.convert_ids_to_tokens(tokenized_data["input_ids"][0])

['[CLS]', 'silver', 'skin', 'laced', 'with', 'his', 'golden', 'blood', '[SEP]']

In [ ]:
first_module = model._first_module()
first_module.auto_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-5): 6 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
embeddings = first_module.auto_model.embeddings
embeddings

BertEmbeddings(
  (word_embeddings): Embedding(30522, 384, padding_idx=0)
  (position_embeddings): Embedding(512, 384)
  (token_type_embeddings): Embedding(2, 384)
  (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [ ]:
device = torch.device("mps" if torch.has_mps else "cpu")
# first_sentence = "there are no rules in the jungle"
# second_sentence = "we learn to make rules among others"

first_sentence = "vector search optimization"
second_sentence = "we learn to apply vector search optimization"

with torch.no_grad():
  first_tokens = model.tokenize(first_sentence)
  second_tokens = model.tokenize(second_sentence)
  first_embeddings = embeddings.word_embeddings(
      first_tokens["input_ids"].to(device)
  )
  second_embeddings = embeddings.word_embeddings(
      second_tokens["input_ids"].to(device)
  )
  first_embeddings = first_embeddings.mean(dim=0)
  second_embeddings = second_embeddings.mean(dim = 0)
  # print("First Token\n", first_tokens)
  # print("Second Token\n", second_tokens)
  # print("First Embeddings\n", first_embeddings)
  # print("Second Embeddings\n", second_embeddings)
print(first_embeddings.shape, second_embeddings.shape)

torch.Size([3, 384]) torch.Size([3, 384])


In [ ]:
from sentence_transformers import util
distances = util.cos_sim(
    first_embeddings.squeeze(),
    second_embeddings.squeeze()
).cpu().numpy()
px.imshow(
    distances,
    x = model.tokenizer.convert_ids_to_tokens(second_tokens["input_ids"][0]),
    y = model.tokenizer.convert_ids_to_tokens(first_tokens["input_ids"][0]),
    text_auto = True,
)

In [ ]:
token_embeddings = first_module.auto_model \
  .embeddings \
  .word_embeddings \
  .weight \
  .detach() \
  .cpu() \
  .numpy()
token_embeddings.shape

(30522, 384)

In [45]:
import random
vocabulary = first_module.tokenizer.get_vocab()
sorted_vocabulary = sorted(
    vocabulary.items(),
    key = lambda x: x[1],
)
sorted_tokens = [ token for token, _ in sorted_vocabulary]
random.choices(sorted_tokens, k = 100)

['irregular',
 'vascular',
 '308',
 'vera',
 'swayed',
 'tipping',
 'sudden',
 'premature',
 'arrangement',
 'queen',
 'ɑ',
 'sunrise',
 'xbox',
 '##g',
 'barrier',
 '।',
 'harmonic',
 'bob',
 'buenos',
 '1300',
 'fray',
 'perished',
 'cleansing',
 'silhouette',
 'finest',
 'indoors',
 'pain',
 'occurred',
 '284',
 'transmitted',
 '2d',
 '##rith',
 'zinc',
 'banana',
 'one',
 '##rst',
 'negro',
 'surveyor',
 'anna',
 'boo',
 'mic',
 'assaults',
 'bind',
 'terrifying',
 'injustice',
 'ice',
 'stating',
 'ow',
 'hayward',
 'shredded',
 'threatening',
 '##outs',
 'milos',
 'depend',
 'ⁱ',
 'elevators',
 '##ood',
 '[unused676]',
 'savior',
 '4th',
 'pulse',
 '州',
 'ヘ',
 '##ラ',
 'soup',
 'points',
 'screens',
 'hatch',
 '##ッ',
 'drought',
 'reich',
 'dismissing',
 'tilting',
 '##mbling',
 'tan',
 'continental',
 'ceiling',
 '[unused47]',
 'ok',
 '##ह',
 'revival',
 'lake',
 '##⊂',
 'turrets',
 '##།',
 'cinder',
 'toad',
 'gigantic',
 'outlet',
 'theologian',
 'simultaneous',
 'farther',
 'r

In [47]:
from sklearn.manifold import TSNE
tsne = TSNE( n_components = 2, metric = "cosine", random_state = 42)
tsne_embeddings_2d = tsne.fit_transform(token_embeddings)
tsne_embeddings_2d.shape

(30522, 2)

In [50]:
token_colors = []
for token in sorted_tokens:
  if token[0] == "[" and token[-1] == "]":
    token_colors.append("red")
  elif token.startswith("##"):
    token_colors.append("blue")
  else:
    token_colors.append("green")

In [51]:
import plotly.graph_objs as go
scatter=go.Scattergl(
    x = tsne_embeddings_2d[:, 0],
    y = tsne_embeddings_2d[:, 1],
    text=sorted_tokens,
    marker=dict(color=token_colors, size =3 ),
    mode = "markers",
    name = "Token embeddings",
)
fig = go.FigureWidget(
    data = [scatter],
    layout = dict(
        width = 600,
        height = 900,
        margin = dict(l =0, r = 0),

    )
)
fig.show()


In [53]:
output_embedding = model.encode(["Silver skin laced with golden blood"])
output_embedding.shape

(1, 384)

In [54]:
output_token_embeddings = model.encode(
    ["Silver skin laced with golden blood"],
    output_value = "token_embeddings"
)
output_token_embeddings[0].shape

torch.Size([8, 384])

In [56]:
first_sent = "You have power over your mind, not outside events"
second_sent = "Power resides within those who know to control their peace of mind"
with torch.no_grad():
  first_token = model.tokenize([first_sent])
  second_token = model.tokenize([second_sent])
  first_embed = model.encode([first_sent],
                             output_value = "token_embeddings")
  second_embed = model.encode([second_sent],
                             output_value = "token_embeddings")

  distances = util.cos_sim(
      first_embed[0],
      second_embed[0]
  )

In [57]:
px.imshow(
    distances.cpu().numpy(),
    x=model.tokenizer.convert_ids_to_tokens(
        second_token["input_ids"][0]
    ),
    y=model.tokenizer.convert_ids_to_tokens(
        first_token["input_ids"][0]
    ),
    text_auto = True,
)